In [6]:
import sys
import io

import os


def UpLoad_File(dir_path):
    """
    遍历给定目录，返回所有文件的完整路径（递归包含子目录）
    自动过滤掉 .DS_Store 等隐藏文件
    """
    file_list = []
    for root, dirs, files in os.walk(dir_path):
        for file in files:
            if not file.endswith('.DS_Store'):
                full_path = os.path.join(root, file)
                file_list.append(full_path)
    return file_list


API_URL = "https://api.siliconflow.cn/v1/chat/completions"
API_KEYS = [
    "sk-dpadryupxccpbkigoduasfosszucawczlmfraqhtevaxlokx",
    "sk-kbucebwhrsoimqttlosgtxncyvmuvdyioncbadavayiovrns",
    "sk-zzuykfebwbxkurfftzuvujqpwqzxxljegnxhhwtzddvwiigf",
    "sk-qgsqryixuqdmtzkgubxpvdzollysgtonnvcrwmikwegmaogn",
    "sk-hvxqvahoplbhdadwtaomisdamxqhquvummcfpvlafeovpqus",
]
INPUT_DIR = '../log/OUTPUT_FILE'
OUTPUT_DIR = '../Find_detect/output_530'
PROMPT_STRATEGIES = 'CoT'
# analyze_log_path = '../Find_detect/output_528'
file_list = UpLoad_File(INPUT_DIR)
# file_list = file_list[:1]   # debug
Results = []
Error_logs = []
for file in file_list:
    INPUT_FILE = file
    raw_file_name = os.path.join(
        OUTPUT_DIR,
        os.path.basename(file).replace('.xlsx', '_raw.xlsx'))
    print(raw_file_name)

../Find_detect/output_530/log_czp_db1_kernel_raw.xlsx
../Find_detect/output_530/log_database_log_raw.xlsx
../Find_detect/output_530/log_log_controller_0_Event_txt_raw.xlsx
../Find_detect/output_530/log_czp_db2_kernel_raw.xlsx
../Find_detect/output_530/log_czp_db2_auth_raw.xlsx
../Find_detect/output_530/log_resampled_sample_txt_raw.xlsx
../Find_detect/output_530/log_czp_db1_auth_raw.xlsx
../Find_detect/output_530/log_czp_db1_messages_raw.xlsx
../Find_detect/output_530/log_czp_db2_messages_raw.xlsx


In [17]:
import pandas

file_name = file_list[-1]
print(file_name)
df = pd.read_excel(file_name).iloc[1:]
df = df.reset_index()
df.head()

../log/OUTPUT_FILE/log_czp_db2_messages.xlsx


,index,log
0,1,Apr 23 00:00:01 czp-db2 /usr/sbin/cron[101839]...
1,2,Apr 23 00:00:01 czp-db2 /usr/sbin/cron[101840]...
2,3,Apr 23 00:00:01 czp-db2 /usr/sbin/cron[101841]...
3,4,Apr 23 00:00:01 czp-db2 /usr/sbin/cron[101842]...
4,5,Apr 23 00:00:01 czp-db2 /usr/sbin/cron[101843]...


In [9]:
import requests
from tenacity import retry, stop_after_attempt, wait_exponential


def run_prompt(prompt: str, api_key: str, api_url: str) -> str:
    """
    调用大模型 API，输入 prompt，返回回答内容。

    参数：
    - prompt: 输入的文本提示
    - api_key: API 密钥（string）
    - api_url: 模型服务的 URL（string）

    返回：
    - 模型输出的内容（string）
    """
    payload = {
        "model": "THUDM/GLM-4-9B-0414",  # 可根据需要替换为 "Qwen/Qwen3-8B" 等模型名
        "stream": False,
        "max_tokens": 8192,
        "enable_thinking": True,
        "thinking_budget": 4096,
        "min_p": 0.05,
        "temperature": 0.1,
        "top_p": 0.3,
        "top_k": 20,
        "frequency_penalty": 0.2,
        "n": 1,
        "stop": [],
        "messages": [{
            "role": "user",
            "content": prompt
        }]
    }
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    response = requests.post(api_url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()['choices'][0]['message']['content']


def build_prompt(df, prompt_header: str, max_len: int = 40000) -> list:
    """
    将 df['log'] 构建为一个或多个符合最大长度限制的 prompt。

    参数：
    - df: 包含日志的 DataFrame，要求含有列 df['log']
    - prompt_header: prompt 开头说明文本（如 log classifier）
    - max_len: 每个 prompt 的最大字符数（默认 4000）

    返回：
    - prompt_list: List[str]，每个元素是一条完整的 prompt
    """
    prompt_list = []
    current_prompt = prompt_header.strip() + "\n"
    current_logs = []
    for _, row in df.iterrows():
        log_entry = f"[{row['index']}] {row['log']}"
        if len(current_prompt) + len(log_entry) + 1 <= max_len:
            current_prompt += log_entry + "\n"
        else:
            prompt_list.append(current_prompt.strip())
            current_prompt = prompt_header.strip() + "\n" + log_entry + "\n"

    # 最后一批日志加入
    if current_prompt.strip() != prompt_header.strip():
        prompt_list.append(current_prompt.strip())
    return prompt_list


In [12]:
import pandas as pd

prompt_header = '''You are a log anomaly classifier.

You will be given a list of log entries, each with a unique index.  
Your task is to determine whether each log is **abnormal (1)** or **normal (0)**.  
Only output in the following format, without any extra explanation or comments.

## Output Format:
(log_idx, status)

## Output Rules:
- status must be 1 if the log entry indicates an error, failure, crash, or unusual behavior.
- status must be 0 if the log entry is a normal operation or informational message.
- Output **only** a list of tuples. No extra text or explanation.

## Input Logs:
'''

prompt_list = build_prompt(df.iloc[:], prompt_header, max_len=8000)
# for prompt in prompt_list:
#     print(prompt)
#     print("\n" + "=" * 80 + "\n")


In [13]:
len(prompt_list)

115

In [14]:
df_1 = df.iloc[:360]
prompt_list = build_prompt(df_1, prompt_header, max_len=6000)
# log_info = run_prompt(prompt_list[0], API_KEYS[0], API_URL)
len(prompt_list)

6

In [313]:
# print(prompt_list[0])

In [314]:
import re


def parse_log_result(raw_output: str) -> dict:
    """
    解析模型输出，如：
    (0, 1)
    (1, 0)
    ...
    返回：{0: 1, 1: 0, ...}
    """
    return {
        int(idx): int(status)
        for idx, status in re.findall(r'\((\d+),\s*(\d+)\)', raw_output)
    }


# 初始化 status 列为空
df_1['status'] = None

all_log_info = []
# 遍历每个 prompt
for i, prompt in enumerate(prompt_list):
    print(f"[{i+1}/{len(prompt_list)}] Running prompt...")
    output = run_prompt(prompt, API_KEYS[-1], API_URL)
    all_log_info.append(output)

/var/folders/jw/phs9hmcj0r9bmsg0v_1pbn4w0000gn/T/ipykernel_4952/1604694894.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['status'] = None


[1/6] Running prompt...
[2/6] Running prompt...
[3/6] Running prompt...
[4/6] Running prompt...
[5/6] Running prompt...
[6/6] Running prompt...


In [315]:
all_info = ""
for i in all_log_info:
    all_info += i
len([i for i in all_info.split("\n") if len(i) > 0])

360

In [317]:
# all_log_info

In [319]:
# [i for i in all_info.split("\n") if len(i) > 0]

In [327]:
df_1['status'] = [
    i.split(",")[-1][:-1].strip() for i in all_info.split("\n") if len(i) > 0
]
df_1

/var/folders/jw/phs9hmcj0r9bmsg0v_1pbn4w0000gn/T/ipykernel_4952/1822134455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['status'] = [


,index,log,status
0,1,Apr 23 14:43:31 czp-db1 kernel: rport-15:0-18...,1
1,2,Apr 23 14:43:31 czp-db1 kernel: lpfc 0000:33:0...,1
2,3,Apr 23 14:43:31 czp-db1 kernel: sd 15:0:3:2: r...,1
3,4,Apr 23 14:43:31 czp-db1 kernel: sd 15:0:3:2: [...,1
4,5,Apr 23 14:43:31 czp-db1 kernel: sd 15:0:3:2: [...,0
...,...,...,...
355,356,Apr 23 15:59:10 czp-db1 kernel: ACPI: LAPIC (a...,0
356,357,Apr 23 15:59:10 czp-db1 kernel: ACPI: LAPIC (a...,0
357,358,Apr 23 15:59:10 czp-db1 kernel: ACPI: LAPIC (a...,0
358,359,Apr 23 15:59:10 czp-db1 kernel: ACPI: LAPIC (a...,0


In [330]:
df_1[df_1['status']=='1']

,index,log,status
0,1,Apr 23 14:43:31 czp-db1 kernel: rport-15:0-18...,1
1,2,Apr 23 14:43:31 czp-db1 kernel: lpfc 0000:33:0...,1
2,3,Apr 23 14:43:31 czp-db1 kernel: sd 15:0:3:2: r...,1
3,4,Apr 23 14:43:31 czp-db1 kernel: sd 15:0:3:2: [...,1
6,7,Apr 23 14:43:31 czp-db1 kernel: end_request: I...,1
7,8,Apr 23 14:43:31 czp-db1 kernel: device-mapper:...,1
11,12,Apr 23 15:59:10 czp-db1 kernel: Cannot find ma...,1
39,40,Apr 23 15:59:10 czp-db1 kernel: AMI BIOS detec...,1


In [235]:
pd.DataFrame([i for i in all_info.split("\n") if len(i) > 0])

,0
0,"(1, 1)"
1,"(2, 1)"
2,"(3, 1)"
3,"(4, 1)"
4,"(5, 0)"
...,...
115,"(116, 0)"
116,"(117, 0)"
117,"(118, 0)"
118,"(119, 0)"


In [215]:
all_log_info[-1]

' '

In [ ]:
# prompt_list[4]

In [ ]:
df[:160]

In [ ]:
log_info = run_prompt(prompt_list[0], API_KEYS[0], API_URL)

In [ ]:
log_info.split("\n")[1:]

In [ ]:
prompt_list[0].split("\n")[-1]

In [ ]:
import re

def extract_index(log_line: str) -> int:
    match = re.search(r'\((\d+)\)', log_line)
    if match:
        return int(match.group(1))
    else:
        return None  # 或 raise ValueError("No index found")

log = prompt_list[0].split("\n")[-1]
index = extract_index(log)
print(index) 


In [ ]:
log_type_list = log_info.split("\n")[1:]

In [ ]:
log_type_list

In [ ]:
import re

status_list = [
    int(re.search(r'\((\d+),\s*(\d+)\)', i).group(2)) for i in log_type_list
]
len(status_list), len()

In [ ]:
# 创建空 status 列，初始化为 NaN
df['status'] = None

In [ ]:
df['status'][:82] = status_list
df[:82]

In [ ]:

import pandas as pd
import re



# 创建空 status 列，初始化为 NaN
df['status'] = None

# 遍历 log_type_list 解析出 index 和 status，并赋值到对应行
for item in log_type_list:
    match = re.match(r'\((\d+),\s*(\d+)\)', item)
    if match:
        idx = int(match.group(1))
        label = int(match.group(2))
        if idx < len(df):  # 防止越界
            df.at[idx, 'status'] = label

df.iloc[:82]

In [ ]:
log_type_list

In [ ]:
prompt_list[0]